In [42]:
import pandas as pd 
import numpy as np
import import_ipynb

from sklearn import tree
from sklearn.impute import SimpleImputer
from functions import *

In [43]:
train= pd.read_csv("../data/raw/aps_failure_training_set.csv")
test = pd.read_csv("../data/raw/aps_failure_test_set.csv")

print('shape of training set is ',train.shape)
print('shape of test set is ',test.shape)

shape of training set is  (60000, 171)
shape of test set is  (16000, 171)


## Data Preprocessing

1. Replace 'na' with null value and convert column into numeric data type
2. Imputing missing variables with mean

In [45]:
###defining target variables(y) and predictor variables (x)
y_train = train['class']
X_train = train.drop('class',axis=1)

y_test = test['class']
X_test = test.drop('class',axis=1)

In [46]:
X_train.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,76698,na,2130706438,280,0,0,0,0,0,0,...,1240520,493384,721044,469792,339156,157956,73224,0,0,0
1,33058,na,0,na,0,0,0,0,0,0,...,421400,178064,293306,245416,133654,81140,97576,1500,0,0
2,41040,na,228,100,0,0,0,0,0,0,...,277378,159812,423992,409564,320746,158022,95128,514,0,0
3,12,0,70,66,0,10,0,0,0,318,...,240,46,58,44,10,0,0,0,4,32
4,60874,na,1368,458,0,0,0,0,0,0,...,622012,229790,405298,347188,286954,311560,433954,1218,0,0


In [47]:
## Replace 'na' with null value and convert column into numeric data type

X_train.replace('na',np.nan,inplace=True)
X_train=X_train[X_train.columns].apply(pd.to_numeric, errors='coerce')

X_test.replace('na',np.nan,inplace=True)
X_test=X_test[X_train.columns].apply(pd.to_numeric, errors='coerce')

X_train.head()

,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,ag_003,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,76698,NaN,2.130706e+09,280.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,33058,NaN,0.000000e+00,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,41040,NaN,2.280000e+02,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,12,0.0,7.000000e+01,66.0,0.0,10.0,0.0,0.0,0.0,318.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,60874,NaN,1.368000e+03,458.0,0.0,0.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [48]:
## imputing mising variables
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

X_train_new_adjusted= pd.DataFrame(imp.fit_transform(X_train),columns=X_train.columns)
X_test_new_adjusted= pd.DataFrame(imp.fit_transform(X_test),columns=X_test.columns)

In [ ]:
X_train_new_adjusted.head()

# Preprocess Function testing

In [49]:
x_train,x_test, y_train, y_test = preprocess(train,test)

x_train.head()
x_test.head()

NameError: name 'preprocess' is not defined

# The super fancy but copy-pastable machine learning

In [ ]:
## Training/Fitting model using training data

clf = tree.DecisionTreeClassifier()
clf.fit(X_train_new_adjusted,y_train)

In [ ]:
## Using model to generate prediction using test data predictor variables

y_pred = clf.predict(X_test_new_adjusted)
y_pred=pd.DataFrame(y_pred)

In [ ]:
##Compute test dataset cost

cost(y_test,y_pred)

# JW's section

## SMOTE

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
smote = SMOTE()

In [ ]:
X_train_smote, y_train_smote = balance_data(X_train_new_adjusted,y_train)
X_test_smote, y_test_smote = balance_data(X_test_new_adjusted,y_test)

In [ ]:
## Testing
clf = tree.DecisionTreeClassifier()
clf.fit(X_train_smote,y_train_smote)
y_pred = clf.predict(X_test_new_adjusted)
y_pred = pd.DataFrame(y_pred)
cost(y_test,y_pred)

# LGBM

In [ ]:
import lightgbm as lgb
from sklearn import metrics

In [ ]:
model= lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model.fit(X_train_smote,y_train_smote,eval_set=[(X_test_new_adjusted,y_test),(X_train_smote,y_train_smote)],
          verbose=20,eval_metric='logloss')